In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### implementation of basic ae

In [59]:
class Evolutionary_Algorithm:
    __slots__ = [
        "mutation_rate",
        "crossover_rate",
        "number_of_generations",
        "population_size",
        "function_to_optimize",
        "population",
        "problem_dim",
        "hall_of_fame",
    ]

    def __init__(
        self,
        mutation_rate=0.7,
        crossover_rate=0.7,
        number_of_generations=50,
        population_size=100,
    ):
        self.population_size = population_size
        self.number_of_generations = number_of_generations
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.function_to_optimize = None
        self.population = None
        self.problem_dim = None
        self.hall_of_fame = []

    def _generate_population(self, n_genes):
        self.population = np.random.uniform(-10, 10, (self.population_size, n_genes))

    def _mutation(self, individual):
        for i in range(len(individual)):
            if np.random.rand() < self.mutation_rate:
                individual += np.random.normal(0, 1, self.problem_dim)
        return individual

    def _crossover(self, parent1, parent2):
        if np.random.rand() < self.crossover_rate:
            crossover_point = np.random.randint(1, self.problem_dim - 1)
            child1 = np.concatenate(
                (parent1[:crossover_point], parent2[crossover_point:])
            )
            child2 = np.concatenate(
                (parent2[:crossover_point], parent1[crossover_point:])
            )
            return child1, child2
        return parent1, parent2

    def _evaluate_population(self):
        fitness = np.zeros(len(self.population))
        for i, individual in enumerate(self.population):
            fitness[i] = self.function_to_optimize(*individual)
        self.hall_of_fame.append(np.argsort(fitness)[: int(self.population_size * 0.1)])
        return fitness

    def _tournament_selection(self, fitness):
        fitness = 1 / fitness
        probabilities = fitness / np.sum(fitness)
        return self.population[
            np.random.choice(range(self.population), p=probabilities)
        ]

    def _select_individual(self):
        return self.population[np.random.choice(range(self.population_size))]

    def _visualize_individual(self, individual):
        print(individual)

    def optimize(self, function_to_optimize):
        # initialize the population
        self.function_to_optimize = function_to_optimize
        self.problem_dim = function_to_optimize.__code__.co_argcount
        self._generate_population(self.problem_dim)

        for generation in range(self.number_of_generations):

            # crossover
            for _ in range(0, self.population_size, 2):
                parent1 = self._select_individual()
                parent2 = self._select_individual()
                child1, child2 = self._crossover(parent1, parent2)
                self.population = np.vstack([self.population, child1])
                self.population = np.vstack([self.population, child2])

            # mutation
            for _ in range(self.population_size):
                individual = self._select_individual()
                self.population = np.vstack(
                    [self.population, self._mutation(individual)]
                )

            # evaluate the population and select the best individuals
            fitness = self._evaluate_population()
            self.population = self._tournament_selection(fitness)
            print(
                f"iteration {generation} best fitness: {np.min(fitness)} for individual {self.population[self.hall_of_fame[-1][0]]}"
            )

### definition of objective functions 

In [60]:
def rastrigin(*args):
    return 10 * len(args) + np.sum(args**2 - 10 * np.cos(2 * np.pi * args))


def rastrigin_5d(x, y, z, w, v):
    return rastrigin(x, y, z, w, v)


def quadratic_3d(x, y, z):
    return x**2 + y**2 + 2 * z**2

In [61]:
ea = Evolutionary_Algorithm()
ea.optimize(quadratic_3d)

TypeError: only integer scalar arrays can be converted to a scalar index